In [1]:
import inspect
import trio

Let's write out own event loop to understand how tings works.

first let's redefine print to print with some indentation

In [90]:
import builtins
level = 0

def print(*args, **kwargs):
    global level
    return builtins.print('    '*level, *args, **kwargs)

In [91]:
print('hey')

 hey


In [94]:
level = 1
print('ho')
level = 0 

     ho


Let's make a decorator that increase/decrease the indentation level of the function it decorate:

In [195]:
def aindent(fun):
    if inspect.iscoroutinefunction(fun):
        async def _wrapper(*args, **kwargs):
            global level
            print('Entering ', fun.__name__)
            level +=1
            res = await fun(*args, **kwargs)
            level -=1
            print('Leaving ', fun.__name__)
            return res
    else:
        # same without async/await
        def _wrapper(*args, **kwargs):
            global level
            print('Entering ', fun.__name__)
            level +=1
            res = fun(*args, **kwargs)
            level -=1
            print('Leaving ', fun.__name__)
            return res
    return _wrapper

In [196]:
@aindent
def toplevel():
    print('> top')
    bottom()
    print('< top')

@aindent
def bottom():
    print('>bottom')

toplevel()

                 Entering  toplevel
                     > top
                     Entering  bottom
                         >bottom
                     Leaving  bottom
                     < top
                 Leaving  toplevel


One misconception (that was up until recently mine), is that most async function call can by asynchronous meaning that every await in your code may be a task switch. 

That is to say in the following, regardless of `async_func` the code **could** be interrupted between assigning to `a`  and assigning to `b`

```python
a = 1
b = await async_func(a)
```

That's not the case, and we'll see that only generator based coroutine can be broken schedule points, and while `async_func` can be pause in the middle of it's execution it's only because it does (maybe indirrectly) call such a function.




## A primer on generator

We'll want to use generator/coroutines to cut our work into tasks we can interrupt, we don't want to `yield` results, but that will help. 

Let's try to buildsuch an example and then manually advance it.

In [197]:
@aindent
async def login(username, pwdhash):
    print('starting login.. check existence')
    exists = await user_exists(username)
    print('exists:', exists)
    if not exists:
        print('Sign in first')
        return
    print('get hash')
    stored_hash =  await get_hash_for(username)
    if stored_hash  == pwdhash:
        print('Come in')
        return
    print('wrong pwd')
    return
    

@aindent    
async def user_exists(username):
    if len(username) <= 8:
        print('User exists  !')
        return True
    print("Your name don't ring a bell.")
    return False

@aindent
async def get_hash_for(username):
    return 'abcd'

In [198]:
coro = login('matthias', 'password')
coro 

<coroutine object aindent.<locals>._wrapper at 0x10a74e990>

So we got a coroutine (think of it as a generator), nothing executeed yet. 
We could call `next()` on a generator, but we actually need to call `.send()` here and send `None` tp start.
Everytime the coroutine can stop it will and will yield a value. It will raise `StopIteration` as soon  as it is done
We'll call `.send()` on it until it raises.

In [199]:
coro.send(None)

                 Entering  login
                     starting login.. check existence
                     Entering  user_exists
                         User exists  !
                     Leaving  user_exists
                     exists: True
                     get hash
                     Entering  get_hash_for
                     Leaving  get_hash_for
                     wrong pwd
                 Leaving  login


StopIteration: 

Hum, it did not pause. That is less than ideal... The reason is you need to use generator base coroutine functions to pause your work, but you should not worry, usually they are provided (and should be provided) y your io library (trio, curio, asyncio...) Let's create our first: `interrupt` that does nothing but pause the execution when it is awaited:

In [206]:

@types.coroutine
def interrupt():
    print('start interrupt')
    yield 'INTERRUPT'
    print('end interrupt')

In [207]:
coro = interrupt()
coro

coro.send(None)
print('====')
coro.send(None)

                 start interrupt
                 ====
                 end interrupt


StopIteration: 

And... let's pause just before getting hashed password from db:

In [233]:
@aindent
async def login(username, pwdhash):
    print('starting login.. check existence')
    exists = await user_exists(username)
    print('exists:', exists)
    if not exists:
        print('Sign in first')
        return
    print('get hash')
    res = await interrupt()
    print('got: ', res)
    stored_hash =  await get_hash_for(username)
    for i,(x,y) in enumerate(zip(stored_hash, pwdhash)):
        await interrupt()
        print('check letter...', i, end='')
        if x != y:
            print('Nop')
            return
        else:
            print('ok')
    print('Come in')
    return

coro = login('matthias', 'abcd')

In [234]:
coro.send(None)

                                 Entering  login
                                     starting login.. check existence
                                     Entering  user_exists
                                         User exists  !
                                     Leaving  user_exists
                                     exists: True
                                     get hash
                                     start interrupt


'INTERRUPT'

Yeah ! First we stopped before checking the password. And we got `INTERRUPT`, so we know we called `interrupt` and can switch task. Let's resume for now.

In [235]:
coro.send(None)

                                     end interrupt
                                     got:  None
                                     Entering  get_hash_for
                                     Leaving  get_hash_for
                                     start interrupt


'INTERRUPT'

In [236]:
coro.send(None)
print('====')
coro.send(None)
print('====')
coro.send(None)

                                     end interrupt
                                     check letter... 0                                     ok
                                     start interrupt
                                     ====
                                     end interrupt
                                     check letter... 1                                     ok
                                     start interrupt
                                     ====
                                     end interrupt
                                     check letter... 2                                     ok
                                     start interrupt


'INTERRUPT'

In [237]:
print('====')
coro.send(None)

                                     ====
                                     end interrupt
                                     check letter... 3                                     ok
                                     Come in
                                 Leaving  login


StopIteration: 

So now, we can write out miniloop:

In [269]:
def miniloop(corofun, *args):
    coro = corofun(*args)

    print('=== starting the coroutine ===')
    coro.send(None)
    print(f'=== started (0) ===')
    for i in range(1, 1000):
        try:
            res = coro.send(i)
        except StopIteration as stahp:
            print('Stahp', stahp)
            return 
        if res == 'INTERRUPT':
            print(f'=== interrupt ({i+1}) ===')
            print('If I had another task I would run it.')
        else:
            print(f'Unknown instruction: {res}')

        
            

In [270]:
level = 0

In [271]:
miniloop(login, 'matthias', 'abcd')

 === starting the coroutine ===
 Entering  login
     starting login.. check existence
     Entering  user_exists
         User exists  !
     Leaving  user_exists
     exists: True
     get hash
     start interrupt
     === started (0) ===
     end interrupt
     got:  None
     Entering  get_hash_for
     Leaving  get_hash_for
     start interrupt
     === interrupt (2) ===
     If I had another task I would run it.
     end interrupt
     check letter... 0     ok
     start interrupt
     === interrupt (3) ===
     If I had another task I would run it.
     end interrupt
     check letter... 1     ok
     start interrupt
     === interrupt (4) ===
     If I had another task I would run it.
     end interrupt
     check letter... 2     ok
     start interrupt
     === interrupt (5) ===
     If I had another task I would run it.
     end interrupt
     check letter... 3     ok
     Come in
 Leaving  login
 Stahp 


In [277]:
import types
import datetime
import time


@types.coroutine
def interrupt():
    return (yield 'SLEEP')

@types.coroutine
def sleep(delta):
    """
    Dummy sleep, will keep yeilding until time has passed
    """

    n= datetime.datetime.now()
    while (datetime.datetime.now() - n).total_seconds() < delta:
        yield 'SLEEP'
        # actually avoid using 100% CPU and leave change to CTRLC.
    return (yield 'RETURNED')
    
counter = 0
def taskgen(name, interval):
    global counter
    counter +=1
    number = counter
    T = 10
    async def task():
        N = 10
        if not callable(interval):
            N = int(T/interval)
            
        for j in range(N):
            if callable(interval):
                res = await sleep(interval())
            else:
                res = await sleep(interval)
            print(' '*10*(number-1), name, number, res)
    return task
        
    
import random
def multiloop(*coroutines_functions):
    coroutines = [c() for c in coroutines_functions]
    for c in coroutines:
        c.send(None)
    i=-1
    while coroutines:
        i+=1
        # now let's randomly advanve one of our coroutines
        c = random.choice(coroutines)
        try:
            # sleep for 0.01 second in case of bug to not suck out all resources.
            time.sleep(0.01)
            res = c.send(i)
        except StopIteration:
            # if our coroutine raise stopiter, it's finished.
            coroutines.remove(c)
        except:
            pass # something really wrong here.
            
            
    

import random
multiloop(taskgen('A', 1), taskgen('B', 3), taskgen('C', lambda : random.randint(1,50)/10))        
    

 === started (0) ===
  A 1 89
  A 1 182
            B 2 252
                      C 3 258
  A 1 265
  A 1 353
  A 1 441
            B 2 504
  A 1 527
                      C 3 544
  A 1 613
  A 1 707
                      C 3 726
            B 2 765
                      C 3 772
  A 1 797
                      C 3 811
  A 1 891
                      C 3 894
                      C 3 928
                      C 3 1027
                      C 3 1220


KeyboardInterrupt: 

In [308]:
pingpong = True

def pp(name, state):
    async def _ping():
        """Sleep randomly print PING, and change value of ping. Loop at most 10 times."""
        global pingpong
        for i in range(10):
            while pingpong is state:
                await sleep(0)
            print(name ,i)
            await sleep(random.randint(5,15)/10)
            pingpong = state

    return _ping


In [311]:
level = 0 
async  def off():
    await sleep(1)
    await taskgen(' TOC', 2)()
multiloop(pp('PING', True), pp('    PONG', False))
          
multiloop(taskgen('TIC', 2), off )

     PONG 0
 === started (0) ===
 PING 0
     PONG 1
 PING 1
     PONG 2
 PING 2
     PONG 3
 PING 3
     PONG 4
 PING 4
     PONG 5
 PING 5
     PONG 6
 PING 6
     PONG 7
 PING 7
     PONG 8
 PING 8
     PONG 9
 PING 9
 === started (0) ===
                                                                                                                          TIC 13 170
                                                                                                                                     TOC 14 258
                                                                                                                          TIC 13 342
                                                                                                                                     TOC 14 430
                                                                                                                          TIC 13 524
                                                                       